In [3]:
from sqlalchemy import MetaData
from sqlalchemy import Table, Column
from sqlalchemy import Integer, String
from sqlalchemy import select

In [4]:
metadata = MetaData()
user_table = Table(
    "user_account",
    metadata,
    Column("id", Integer, primary_key=True),
    Column("username", String(50), nullable=False),
    Column("fullname", String(255)),
)

In [5]:
user_table.name

'user_account'

In [6]:
user_table.c.username

Column('username', String(length=50), table=<user_account>, nullable=False)

In [10]:
print(user_table.c)

ReadOnlyColumnCollection(user_account.id, user_account.username, user_account.fullname)


In [11]:
print(user_table.c.username.name)
print(user_table.c.username.type)

username
VARCHAR(50)


In [12]:
user_table.primary_key

PrimaryKeyConstraint(Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False))

In [15]:
# Table object is the core of SQL expression.
print(select(user_table))

SELECT user_account.id, user_account.username, user_account.fullname 
FROM user_account


In [18]:
from sqlalchemy import create_engine

# Create a database in memory.
engine = create_engine("sqlite://", future=True, echo=True)

with engine.begin() as conn:
    metadata.create_all(conn)

2023-06-23 20:38:09,640 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-23 20:38:09,641 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2023-06-23 20:38:09,641 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-23 20:38:09,642 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2023-06-23 20:38:09,643 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-23 20:38:09,643 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	username VARCHAR(50) NOT NULL, 
	fullname VARCHAR(255), 
	PRIMARY KEY (id)
)


2023-06-23 20:38:09,644 INFO sqlalchemy.engine.Engine [no key 0.00037s] ()
2023-06-23 20:38:09,644 INFO sqlalchemy.engine.Engine COMMIT


In [19]:
from sqlalchemy import String, Numeric, DateTime, Enum

fancy_table = Table(
    "fancy",
    metadata,
    Column("key", String(50), primary_key=True),
    Column("timestamp", DateTime),
    Column("amount", Numeric(10, 2)),
    Column("type", Enum("a", "b", "c")),
)

with engine.begin() as conn:
    fancy_table.create(conn)

2023-06-23 20:39:18,732 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-23 20:39:18,733 INFO sqlalchemy.engine.Engine 
CREATE TABLE fancy (
	"key" VARCHAR(50) NOT NULL, 
	timestamp DATETIME, 
	amount NUMERIC(10, 2), 
	type VARCHAR(1), 
	PRIMARY KEY ("key")
)


2023-06-23 20:39:18,734 INFO sqlalchemy.engine.Engine [no key 0.00037s] ()
2023-06-23 20:39:18,734 INFO sqlalchemy.engine.Engine COMMIT


In [22]:
print(metadata.tables.keys())
metadata.tables['user_account']

dict_keys(['user_account', 'fancy'])


Table('user_account', MetaData(), Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False), Column('username', String(length=50), table=<user_account>, nullable=False), Column('fullname', String(length=255), table=<user_account>), schema=None)

In [23]:
from sqlalchemy import ForeignKey

addresses_table = Table(
    "email_address",
    metadata,
    Column("id", Integer, primary_key=True),
    Column("email_address", String(100), nullable=False),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
)

with engine.begin() as conn:
    addresses_table.create(conn)

2023-06-23 20:43:36,209 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-23 20:43:36,210 INFO sqlalchemy.engine.Engine 
CREATE TABLE email_address (
	id INTEGER NOT NULL, 
	email_address VARCHAR(100) NOT NULL, 
	user_id INTEGER NOT NULL, 
	PRIMARY KEY (id), 
	FOREIGN KEY(user_id) REFERENCES user_account (id)
)


2023-06-23 20:43:36,211 INFO sqlalchemy.engine.Engine [no key 0.00047s] ()
2023-06-23 20:43:36,212 INFO sqlalchemy.engine.Engine COMMIT


In [29]:
from sqlalchemy import ForeignKeyConstraint
from sqlalchemy import Text

story_table = Table(
    "story",
    metadata,
    Column("story_id", Integer, primary_key=True),
    Column("version_id", Integer, primary_key=True),
    Column("headline", String(100), nullable=False),
    Column("body", Text),
)

published_table = Table(
    "published",
    metadata,
    Column("pub_id", Integer, primary_key=True),
    Column("pub_timestamp", DateTime, nullable=False),
    Column("story_id", Integer),
    Column("version_id", Integer),
    ForeignKeyConstraint(
        ["story_id", "version_id"], ["story.story_id", "story.version_id"]
    ),
)

with engine.begin() as conn:
    metadata.create_all(conn)

2023-06-23 20:52:32,129 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-23 20:52:32,130 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2023-06-23 20:52:32,130 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-23 20:52:32,131 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("fancy")
2023-06-23 20:52:32,132 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-23 20:52:32,133 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("email_address")
2023-06-23 20:52:32,133 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-23 20:52:32,133 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("story")
2023-06-23 20:52:32,134 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-23 20:52:32,134 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("story")
2023-06-23 20:52:32,135 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-23 20:52:32,136 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("published")
2023-06-23 20:52:32,136 INFO sqlalchemy.engine.Engine [raw 

In [24]:
# Reflections (loading tables by reading from the database).
metadata2 = MetaData()
with engine.connect() as conn:
    user_reflected = Table("user_account", metadata2, autoload_with=conn)

2023-06-23 20:47:14,315 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-23 20:47:14,316 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("user_account")
2023-06-23 20:47:14,316 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-23 20:47:14,317 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type in ('table', 'view')
2023-06-23 20:47:14,317 INFO sqlalchemy.engine.Engine [raw sql] ('user_account',)
2023-06-23 20:47:14,318 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("user_account")
2023-06-23 20:47:14,319 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-23 20:47:14,319 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("user_account")
2023-06-23 20:47:14,320 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-23 20:47:14,320 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AN

In [28]:
print(user_reflected.c)
print(user_reflected.primary_key)
print(select(user_reflected))
print(metadata2.tables.keys())

ReadOnlyColumnCollection(user_account.id, user_account.username, user_account.fullname)
PrimaryKeyConstraint(Column('id', INTEGER(), table=<user_account>, primary_key=True, nullable=False))
SELECT user_account.id, user_account.username, user_account.fullname 
FROM user_account
dict_keys(['user_account'])


In [30]:
# Inspect information about tables at a more specific level.
from sqlalchemy import inspect

inspector = inspect(engine)

In [32]:
inspector.get_table_names()

2023-06-23 20:53:52,107 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-23 20:53:52,108 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2023-06-23 20:53:52,108 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-23 20:53:52,109 INFO sqlalchemy.engine.Engine ROLLBACK


['email_address', 'fancy', 'published', 'story', 'user_account']

In [34]:
inspector.get_columns("email_address")

2023-06-23 20:54:20,568 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-23 20:54:20,568 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("email_address")
2023-06-23 20:54:20,569 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-23 20:54:20,569 INFO sqlalchemy.engine.Engine ROLLBACK


[{'name': 'id',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'primary_key': 1},
 {'name': 'email_address',
  'type': VARCHAR(length=100),
  'nullable': False,
  'default': None,
  'primary_key': 0},
 {'name': 'user_id',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'primary_key': 0}]

In [35]:
inspector.get_foreign_keys("email_address")

2023-06-23 20:56:00,797 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-23 20:56:00,798 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("email_address")
2023-06-23 20:56:00,798 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-23 20:56:00,799 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type in ('table', 'view')
2023-06-23 20:56:00,799 INFO sqlalchemy.engine.Engine [raw sql] ('email_address',)
2023-06-23 20:56:00,800 INFO sqlalchemy.engine.Engine ROLLBACK


[{'name': None,
  'constrained_columns': ['user_id'],
  'referred_schema': None,
  'referred_table': 'user_account',
  'referred_columns': ['id'],
  'options': {}}]

In [41]:
# Reflect an entire schema (reflect all the tables at once).
metadata3 = MetaData()
with engine.connect() as conn:
    metadata3.reflect(conn)

2023-06-23 21:05:10,987 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-23 21:05:10,988 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2023-06-23 21:05:10,988 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-23 21:05:10,989 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_temp_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2023-06-23 21:05:10,990 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-23 21:05:10,990 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("email_address")
2023-06-23 21:05:10,991 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-23 21:05:10,991 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("fancy")
2023-06-23 21:05:10,992 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-23 21:05:10,992 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("published")
2023-06-23 21:05:10,992 INFO sqlalchemy.engine.Engine [raw sql] (

In [38]:
story, published = metadata3.tables["story"], metadata3.tables["published"]

In [39]:
story

Table('story', MetaData(), Column('story_id', INTEGER(), table=<story>, primary_key=True, nullable=False), Column('version_id', INTEGER(), table=<story>, primary_key=True, nullable=False), Column('headline', VARCHAR(length=100), table=<story>, nullable=False), Column('body', TEXT(), table=<story>), schema=None)

In [40]:
print(select(story).join(published))

SELECT story.story_id, story.version_id, story.headline, story.body 
FROM story JOIN published ON story.story_id = published.story_id AND story.version_id = published.version_id
